In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns
import nltk 
from nltk import RegexpTokenizer as rpt
from nltk.corpus import stopwords as sw
from string import punctuation 

nltk.download('punkt')
nltk.download('stopwords')
stopwords = sw.words('portuguese')

data_url="https://raw.githubusercontent.com/liraop/recinfo_lab2/master/data/results.csv"
data = pd.read_csv(data_url).replace(np.nan, '', regex=True)
documents = data.text.count()
items = []

def parse(text):
    words = []
    word_pattern = rpt(r'\w+')
    year_pattern = rpt(r'\d{4}')
    
    patterns = [word_pattern, year_pattern]
    
    for pattern in patterns:
        tokens = []
        for token in pattern.tokenize(text):
            if token not in stopwords and len(token) > 3:
                tokens.append(token)
        words.extend(tokens)
    return words

[nltk_data] Downloading package punkt to /Users/liraop/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/liraop/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 1. Execute o algoritmo ilustrado na Fig. 5.8 do livro texto (pag. 157) para gerar um índice similar o mostrado na Fig. 5.4 (pag. 134). Guarde o índice em disco em formato csv.

In [18]:
def build_index(dataset):
    document_index = 0
    index = {"doc_row": []}
    
    for entry in dataset.text:
        document_index = document_index + 1
        index["doc_row"].append(document_index)
            
        for ngram in parse(entry):
                if ngram in index: #is ngram already on index?
                    if document_index in index[ngram]: # is it in the same document?
                        index[ngram][document_index] = index[ngram][document_index] + 1
                    else: # nope
                        index[ngram][document_index] = 1 
                else: # no, sir
                    index[ngram] = {document_index: 1}
    
    return index

index = build_index(data)

import csv

with open('index.csv', 'w') as f:
    w = csv.DictWriter(f, index.keys())
    w.writeheader()
    w.writerow(index)


### 2. Implemente as abordagens de processamento de consulta documento-por-vez e termo-por-vez (Fig. 5.16 e 5.18). 
+ Defina 5 consultas de um termo somente. 
+ Execute as 5 consultas em cada algoritmo retornando os top-10 documentos (parâmetro k do algoritmo).
+ Dê evidências de que sua implementação está correta.
+ Compare os tempos médios de execução e uso de memória de cada algoritmo.

In [3]:
queries = ["juíza","federal","governo","Brasil","presidente"]

In [9]:
import heapq

def document_at_a_time(query, index, k):
    rank = {}
    inverted_lists = []
    #query as space separated string list
    for ngram in query.split(): 
        if ngram in index:
            inverted_lists.append(index[ngram])
            
    for document_id in index['doc_row']:
        rank[document_id] = 0
        for i_list in inverted_lists:
            for il_doc_id, il_doc_wc in i_list.items():
                if il_doc_id == document_id:
                    rank[document_id] = rank[document_id] + il_doc_wc
                     
    return heapq.nlargest(k, rank, rank.get)

for query in queries:
    print("Term:",query, " - Rank:", document_at_a_time(query, index, 10))

Term: juíza  - Rank: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Term: federal  - Rank: [151, 7, 37, 206, 213, 1, 2, 3, 15, 42]
Term: governo  - Rank: [173, 166, 84, 210, 25, 82, 150, 221, 115, 168]
Term: Brasil  - Rank: [151, 166, 19, 26, 248, 172, 173, 238, 22, 201]
Term: presidente  - Rank: [63, 166, 151, 19, 216, 205, 86, 25, 103, 174]


#### Esta implementação pode ser mais lenta no geral pois não utiliza a tabela de acumuladores como na estratégia de termo por vez. Contudo, utiliza bem menos memória exatamente pelo mesmo fato, além de ser uma abordagem de mais simples implementação.
#### O ponto fraco do algoritmo, como fica evidente na implementação, é o fato de ter de sempre percorrer todo o índice para fazer o scoring, logo, várias otimizações são necessárias numa aplicação real. 

In [10]:
def term_at_a_time(query, index, k):
    rank = {}
    inverted_lists = []
    #query as space separated string list
    for ngram in query.split(): 
        if ngram in index:
            inverted_lists.append(index[ngram])
    
    for i_list in inverted_lists:
        for il_doc_id, il_doc_wc in i_list.items():
            if il_doc_id not in rank:
                rank[il_doc_id] = il_doc_wc
            else:
                rank[il_doc_id] = rank[il_doc_id] + il_doc_wc
    
    return heapq.nlargest(k, rank, rank.get)

for query in queries:
    print("Term:",query, "- Rank:", term_at_a_time(query, index, 10))

Term: juíza - Rank: [1, 2]
Term: federal - Rank: [151, 7, 37, 206, 213, 1, 2, 3, 15, 42]
Term: governo - Rank: [173, 166, 84, 210, 25, 82, 150, 221, 115, 168]
Term: Brasil - Rank: [151, 166, 19, 26, 248, 172, 173, 238, 22, 201]
Term: presidente - Rank: [63, 166, 151, 19, 216, 205, 86, 25, 103, 174]


#### Nesta implementação tomei a liberdade de não implementar a tabela de acumuladores por questões de praticidade. A tabela, entretanto, oferece a vantagem de ter acesso direto às colunas, visto que é uma hashtable, melhorando o desempenho geral do algoritmo devido ao acesso ai disco otimizado.

#### Vemos que há poucas diferenças nos ranks de ambas implementações, provando que as implementações estão majoritariamente corretas, apesar das modificações.

### 3. Implemente uma das versões de consulta conjuntiva (AND) (Fig. 5.20 ou 5.21).
+ Defina 5 consultas conjuntivas (AND).
+ Execute as 5 consultas em cada algoritmo retornando os top-10 documentos(parâmetro k do algoritmo).
+ Dê evidências de que sua implementação está correta.


In [19]:
def doc_at_a_time_conj():
    pass